<a href="https://colab.research.google.com/github/SebastianJia/nlp_research_conceptor/blob/master/LDT_BATS_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDT BATS with CN

In [0]:
#Install vecto library

In [1]:
!pip install vecto

    100% |████████████████████████████████| 81kB 3.2MB/s 
    100% |████████████████████████████████| 450kB 25.3MB/s 
    100% |████████████████████████████████| 71kB 28.3MB/s 
    100% |████████████████████████████████| 81kB 30.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/bd/e6/74/5f9ff2150a62868acf69169db535d99949f838ac2f8e0b5d4b
Successfully built py-cpuinfo


In [0]:
#Install libraries to vecto requirement

In [2]:
!pip install gitpython
!pip install progressbar2

In [0]:
#Import libraries

In [3]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')
#import brewer2mpl
import chainer
import tables
import traitlets
import tqdm
import keras
from numpy.linalg import norm, inv, eig

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
#Get Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download). They used 300d vectors in their paper.

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-02-08 20:58:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-02-08 20:58:24--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  3.07MB/s    in 8m 6s   

2019-02-08 21:06:30 (1.69 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
!mkdir /content/embeddings/
!mv /content/glove.6B.300d.txt /content/embeddings/

In [0]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [0]:
#Convert txt data to word2vec format

In [8]:
!python -m gensim.scripts.glove2word2vec -i /content/embeddings/glove.6B.300d.txt -o /content/glove.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-02-08 21:09:27,128 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py -i /content/embeddings/glove.6B.300d.txt -o /content/glove.txt
2019-02-08 21:09:27,815 - glove2word2vec - INFO - converting 400000 vectors from /content/embeddings/glove.6B.300d.txt to /content/glove.txt
2019-02-08 21:09:35,849 - glove2word2vec - INFO - Converted model with 400000 vectors and 300 dimensions


In [0]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'glove.txt', binary=False)

In [0]:
#Conceptor Negation(Inputs: name string of word vector model(e.g 'glove'), alpha =1 or 2)

In [0]:
def protoConceptor(wordVecModel_str, alpha):
    # compute the prototype conceptor with alpha = 1
    
    
    wordVecModel = eval(wordVecModel_str)    



    # put the word vectors in columns
    x_collector = wordVecModel.vectors[:].T       
        
    num_vec = x_collector.shape[0]
    nrWords = x_collector.shape[1] # number of total words
    
    
    R = x_collector.dot(x_collector.T) / nrWords # calculate the correlation matrix
    
    C = R @ inv(R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    mat = ((np.eye(num_vec)-C)@ x_collector).T
    return  mat

In [0]:
cn_f_name = 'glove'
glove_cn_mat = protoConceptor(cn_f_name, 1)

In [0]:
#Write CN word vectors into a txt file, which will be read by vecto library for calculation. Might take a few minutes.

In [0]:
!mkdir /content/cn_embeddings/

In [0]:
model = eval('glove')
f = open('/content/cn_embeddings/glove_cn.txt','w+')
for word in model.vocab:
  f.write(word+' ')
  id = model.vocab[word].index
  for item in glove_cn_mat[id,:]:
    f.write(str(item)+' ')
  f.write('\n')
f.close()

# Load BATS datasets

In [0]:
#Download BATS dataset

In [14]:
!gdown https://drive.google.com/uc?id=14XcqPQZDlP5gMWAb4tFhTz16snJR73ly


Downloading...
From: https://drive.google.com/uc?id=14XcqPQZDlP5gMWAb4tFhTz16snJR73ly
To: /content/BATS_3.0.zip
100% 43.5k/43.5k [00:00<00:00, 18.4MB/s]


In [15]:
!unzip BATS_3.0.zip

Archive:  BATS_3.0.zip
   creating: BATS_3.0/
   creating: BATS_3.0/4_Lexicographic_semantics/
  inflating: BATS_3.0/4_Lexicographic_semantics/L03 [hyponyms - misc].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L02 [hypernyms - misc].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L01 [hypernyms - animals].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L09 [antonyms - gradable].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L06 [meronyms - part].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L07 [synonyms - intensity].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L08 [synonyms - exact].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L10 [antonyms - binary].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L04 [meronyms - substance].txt  
  inflating: BATS_3.0/4_Lexicographic_semantics/L05 [meronyms - member].txt  
   creating: BATS_3.0/3_Encyclopedic_semantics/
  inflating: BATS_3.0/3_Encyclopedic_semantics/E02 [country - language].t

# Results

Analogy results using LRCos method. The method will look for top 5 neighbors but only count the first neighbor into accuracy. \
The results would be slightly different for each run

In [0]:
import vecto
#import vsmlib
from vecto.embeddings import load_from_dir
from vecto.benchmarks.analogy import Analogy

In [0]:
#Get BATS results(Inputs: fil_dir1 is the directory of embeddings, fil_dir2 is the directory of BATS dataset)

In [0]:
def bats_results(file_dir1, file_dir2):
  path_model = file_dir1
  model = load_from_dir(path_model)
  options = {}
  options["path_dataset"] = file_dir2
  options["name_method"] = "LRCos"  #name of the method, default value is 3CosAdd
  analogy = Analogy(method = options["name_method"], do_top5 = True)
  results = analogy.get_result(model, options["path_dataset"])

  return results

In [0]:
# Their code calculates accuracy in a cumulative way, so the average accuracy is the 10th number.(10 files in each category)

In [20]:
acc_lexico = bats_results('/content/embeddings/','/content/BATS_3.0/4_Lexicographic_semantics/')
accu_lexico = acc_lexico[9]['result']['accuracy']
acc_inf = bats_results('/content/embeddings/','/content/BATS_3.0/1_Inflectional_morphology/')
accu_inf = acc_inf[9]['result']['accuracy']
acc_deriv = bats_results('/content/embeddings/','/content/BATS_3.0/2_Derivational_morphology/')
accu_deriv = acc_deriv[9]['result']['accuracy']
acc_en = bats_results('/content/embeddings/','/content/BATS_3.0/3_Encyclopedic_semantics/')
accu_en = acc_en[9]['result']['accuracy']

INFO:vecto.embeddings:/content/embeddings/Detected VSM in plain text format
INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L01 [hypernyms - animals].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L02 [hypernyms - misc].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L03 [hyponyms - misc].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L04 [meronyms - substance].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L05 [meronyms - member].txt
1

In [0]:
#Print average accuracy of each category. 

In [21]:
print('Raw lexicograpic top1 accuracy: ', accu_lexico)
print('Raw inflectional top1 accuracy: ',accu_inf)
print('Raw derivational top1 accuracy: ', accu_deriv)
print('Raw encyclopedic top1 accuracy: ', accu_en)

Raw lexicograpic top1 accuracy:  0.2560483870967742
Raw inflectional top1 accuracy:  0.7855711422845691
Raw derivational top1 accuracy:  0.29568788501026694
Raw encyclopedic top1 accuracy:  0.4497991967871486


In [0]:
#CN results 

In [25]:
acc_lexico = bats_results('/content/cn_embeddings/','/content/BATS_3.0/4_Lexicographic_semantics/')
accu_lexico_cn = acc_lexico[9]['result']['accuracy']
acc_inf = bats_results('/content/cn_embeddings/','/content/BATS_3.0/1_Inflectional_morphology/')
accu_inf_cn = acc_inf[9]['result']['accuracy']
acc_deriv = bats_results('/content/cn_embeddings/','/content/BATS_3.0/2_Derivational_morphology/')
accu_deriv_cn = acc_deriv[9]['result']['accuracy']
acc_en = bats_results('/content/cn_embeddings/','/content/BATS_3.0/3_Encyclopedic_semantics/')
accu_en_cn = acc_en[9]['result']['accuracy']

INFO:vecto.embeddings:/content/cn_embeddings/Detected VSM in plain text format
INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L01 [hypernyms - animals].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L02 [hypernyms - misc].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L03 [hyponyms - misc].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L04 [meronyms - substance].txt
100% (50 of 50) |########################| Elapsed Time: 0:00:28 ETA:  00:00:00INFO:vecto.benchmarks.analogy.analogy:processing /content/BATS_3.0/4_Lexicographic_semantics/L05 [meronyms - member].tx

In [26]:
print('CN lexicograpic top1 accuracy: ', accu_lexico_cn)
print('CN inflectional top1 accuracy: ',accu_inf_cn)
print('CN derivational top1 accuracy: ', accu_deriv_cn)
print('CN encyclopedic top1 accuracy: ', accu_en_cn)

CN lexicograpic top1 accuracy:  0.2439516129032258
CN inflectional top1 accuracy:  0.8176352705410822
CN derivational top1 accuracy:  0.32854209445585214
CN encyclopedic top1 accuracy:  0.46586345381526106
